# Random Forest

## Prior Progress
Random Forest 14 got 70% accuracy on 5 classes (we didn't have class 0 yet).

## Next Task
Do the 6-way classification using Nucleus plus RBC.

In [1]:
import datetime
print(datetime.datetime.now())
from platform import python_version
print('Python',python_version())
import numpy as np
import pandas as pd
import sklearn   # pip install --upgrade scikit-learn
print('sklearn',sklearn.__version__)
import tensorflow as tf
tf.config.list_physical_devices('GPU')

2022-06-07 09:39:52.024409
Python 3.8.10
sklearn 1.0.2


[PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.utils import shuffle
from sklearn.feature_selection import RFE
import joblib # used to dump/load sklearn models

In [3]:
BASE_DIR='/home/jrm/ShepherdML/TumorII/'
NUCLEUS_FILES=[
    'Nucleus_Rollup_0.csv',
    'Nucleus_Rollup_1.csv',
    'Nucleus_Rollup_2.csv',
    'Nucleus_Rollup_3.csv',
    'Nucleus_Rollup_4.csv',
    'Nucleus_Rollup_5.csv']
RBC_FILES=[
    'RBC_Rollup_0.csv',
    'RBC_Rollup_1.csv',
    'RBC_Rollup_2.csv',
    'RBC_Rollup_3.csv',
    'RBC_Rollup_4.csv',
    'RBC_Rollup_5.csv']
CLASSES=range(0,6)   
MODELS_DIR='/home/jrm/Adjeroh/Naved/models/RandomForest.15/'

In [4]:
print(datetime.datetime.now())
PREFIX='abc_'
def set_prefix(prefix):
    global PREFIX
    PREFIX=prefix
def get_prefix(col_name):
    global PREFIX
    return PREFIX+col_name  # hard coded for now
def load_all_classes():
    X = None
    y = None
    for i in CLASSES:
        print(datetime.datetime.now())
        print("Loading class",i)
        Xnuc = pd.read_csv(BASE_DIR+NUCLEUS_FILES[i])
        Xnuc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Nuc_')
        Xnuc.rename(get_prefix,axis='columns',inplace=True)
        Xrbc = pd.read_csv(BASE_DIR+RBC_FILES[i])
        Xrbc.set_index(['PatchNumber'],inplace=True)
        set_prefix('Rbc_')
        Xrbc.rename(get_prefix,axis='columns',inplace=True)
        Xi = Xnuc.join(Xrbc,how='outer')
        #Xi.drop(columns=['PatchNumber'],inplace=True)
        size = len(Xi)
        yi = np.ones(size) * i   # e.g. class 3
        if X is None:
            X = Xi
            y = yi
        else:
            X = pd.concat( (X,Xi) )
            y = np.concatenate( (y,yi) )
    X.fillna(0,inplace=True)  
    return X,y
Xtrain,ytrain=load_all_classes()
Xtrain

2022-06-07 09:39:54.100346
2022-06-07 09:39:54.100569
Loading class 0


FileNotFoundError: [Errno 2] No such file or directory: '/home/jrm/ShepherdML/TumorII/Nucleus_Rollup_0.csv'

In [ ]:
# RandomForestClassifier can only track feature names of type string.
num_problems=0
for name in Xtrain.columns:
    if not isinstance(name,str):
        num_problems += 1
        print(type(name),name)
if num_problems==0:
    print("Ok")

In [ ]:
print(datetime.datetime.now())
class RF_Util:
    def __init__(self):
        self.model=RandomForestClassifier()
    def get_model(self):
        return self.model
    def set_train(self,X,y):
        self.Xtr = X
        self.ytr = y
    def set_validation(self,X,y):
        self.Xval = X
        self.yval = y
    def fit(self):
        self.model=RandomForestClassifier()
        self.model.fit(self.Xtr,self.ytr)
    def cross_validation(self,fold=5):
        self.model=RandomForestClassifier()
        cv_scores = cross_val_score(self.model, self.Xtr, self.ytr, cv=fold)
        return cv_scores
    def validation_accuracy(self):
        ypred = self.model.predict(self.Xval)
        matches = np.count_nonzero(self.yval==ypred)
        accuracy = 100.0 * matches / len(ypred)  
        return accuracy
    def validation_confusion(self):
        ypred = self.model.predict(self.Xval)
        cm = confusion_matrix(self.yval, ypred)
        return cm
    def important_features(self):
        names = self.model.feature_names_in_
        importances = self.model.feature_importances_
        pairs = np.column_stack( (names,importances) )
        top_array = sorted(pairs, key = lambda e:e[1], reverse=True)
        # There must be a way to do this witout a loop!
        top_list = []
        for i in top_array:
             top_list.append((i[1],i[0]))  # 0=feature_name, 1=importance
        top_df = pd.DataFrame(top_list)
        return top_df

In [ ]:
print(datetime.datetime.now())
print('Do a one-time train/valid split...')
Xt,Xv,yt,yv = train_test_split(Xtrain, ytrain.ravel()) 
        # ,random_state=42) # add this for reproducibility
print('Xtrain',Xt.shape,'ytrain',yt.shape,'non-zero:',np.count_nonzero(yt))
print('Xvalid',Xv.shape,'yvalid',yv.shape,'non-zero:',np.count_nonzero(yv))
print('Do a one-time fit and evaluate...')
rf1 = RF_Util()
rf1.set_train(Xt,yt)
rf1.set_validation(Xv,yv)
print(datetime.datetime.now())
rf1.fit()
print(datetime.datetime.now())
print('Accuracy:',rf1.validation_accuracy())
print('Confusion:')
print(rf1.validation_confusion())

In [ ]:
Xt,Xv,yt,yv=None,None,None,None
rf1=None

In [ ]:
print(datetime.datetime.now())
print("Shuffle...")
# Shuffle (happens automatically with cross validation but not with fit())
Xtrain,ytrain=shuffle(Xtrain,ytrain)  #set random_state for reproducibility

In [ ]:
print(datetime.datetime.now())
print('Cross-validation...')
rf2 = RF_Util()
rf2.set_train(Xtrain,ytrain)
cv_scores = rf2.cross_validation()
print(cv_scores)
print('mean %.4f +/- %.4f' % (cv_scores.mean(),cv_scores.std()))

In [ ]:
rf2=None

In [ ]:
print(datetime.datetime.now())
print('Re-train on full training set...')
rf3 = RF_Util()
rf3.set_train(Xtrain,ytrain)
rf3.fit()
print(datetime.datetime.now())
print('...and rank the features by importance.')
top = rf3.important_features()

In [ ]:
pd.set_option('display.max_rows', None)
top.loc[:25]

In [ ]:
print(datetime.datetime.now())